<a href="https://colab.research.google.com/github/Aanoush-Surana/Machine_Learning/blob/main/DRAFT_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GPU

In [ ]:
!nvidia-smi

!pip install -U torch torchvision torchaudio
!pip install -U 'git+https://github.com/facebookresearch/detectron2.git'

Tue Feb  3 13:04:09 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

MOUNT DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


DEFINE PATHS

In [ ]:

import os

# Your data in Google Drive
IMG_ROOT = '/content/drive/MyDrive/idd20kII/leftImg8bit/train'
GT_ROOT = '/content/drive/MyDrive/idd20kII/gtFine/train'

# Output directories (in Colab temporary storage)
PAN_ROOT = '/content/idd_panoptic/train'
SEM_ROOT = '/content/idd_semantic/train'
JSON_OUT = '/content/panoptic_train.json'
OUTPUT_DIR = '/content/output'

# Create all directories
for path in [PAN_ROOT, SEM_ROOT, OUTPUT_DIR]:
    os.makedirs(path, exist_ok=True)

# Processing limit (set to None for full dataset)
LIMIT = 50

print("="*70)
print("PATHS CONFIGURED")
print("="*70)
print(f"Images:    {IMG_ROOT}")
print(f"GT JSON:   {GT_ROOT}")
print(f"Panoptic:  {PAN_ROOT}")
print(f"Semantic:  {SEM_ROOT}")
print(f"JSON:      {JSON_OUT}")
print(f"Output:    {OUTPUT_DIR}")
print(f"Limit:     {LIMIT}")
print("="*70)

PATHS CONFIGURED
Images:    /content/drive/MyDrive/idd20kII/leftImg8bit/train
GT JSON:   /content/drive/MyDrive/idd20kII/gtFine/train
Panoptic:  /content/idd_panoptic/train
Semantic:  /content/idd_semantic/train
JSON:      /content/panoptic_train.json
Output:    /content/output
Limit:     50


VERIFY

In [ ]:
print("\n" + "="*70)
print("CHECKING YOUR DATA")
print("="*70)

# Check images
if os.path.isdir(IMG_ROOT):
    img_cities = [d for d in os.listdir(IMG_ROOT) if os.path.isdir(os.path.join(IMG_ROOT, d))]
    print(f"\n✓ Images: {len(img_cities)} cities found")
    if img_cities:
        sample = os.path.join(IMG_ROOT, img_cities[0])
        imgs = [f for f in os.listdir(sample) if f.endswith('.jpg')]
        print(f"  Example: {img_cities[0]} has {len(imgs)} images")
        if imgs:
            print(f"  Sample: {imgs[0]}")
else:
    raise FileNotFoundError(f"IMG_ROOT not found: {IMG_ROOT}")

# Check GT JSON files
if os.path.isdir(GT_ROOT):
    gt_cities = [d for d in os.listdir(GT_ROOT) if os.path.isdir(os.path.join(GT_ROOT, d))]
    print(f"\n✓ Ground Truth: {len(gt_cities)} cities found")
    if gt_cities:
        sample = os.path.join(GT_ROOT, gt_cities[0])
        jsons = [f for f in os.listdir(sample) if f.endswith('_polygons.json')]
        print(f"  Example: {gt_cities[0]} has {len(jsons)} JSON files")
        if jsons:
            print(f"  Sample: {jsons[0]}")
else:
    raise FileNotFoundError(f"GT_ROOT not found: {GT_ROOT}")

print("="*70)


CHECKING YOUR DATA

✓ Images: 249 cities found
  Example: 201 has 11 images
  Sample: frame1979_leftImg8bit.jpg

✓ Ground Truth: 249 cities found
  Example: 203 has 24 JSON files
  Sample: frame2605_gtFine_polygons.json


CREATE PANOPTIC + SEMANTIC MASKS

In [ ]:
import json
import numpy as np
from PIL import Image
import cv2

print("\n" + "="*70)
print("CREATING PANOPTIC & SEMANTIC MASKS FROM POLYGON JSON")
print("="*70)

def polygon_to_mask(polygons, img_height, img_width, instance_id):
    """
    Convert polygon coordinates to a binary mask with given instance_id.

    Args:
        polygons: List of polygon coordinates [[x1,y1,x2,y2,...], ...]
        img_height, img_width: Image dimensions
        instance_id: Unique ID for this instance

    Returns:
        mask: numpy array with instance_id where polygon is, 0 elsewhere
    """
    mask = np.zeros((img_height, img_width), dtype=np.int32)

    for poly in polygons:
        if len(poly) < 6:  # Need at least 3 points (6 coordinates)
            continue

        # Reshape to [(x,y), (x,y), ...] format
        points = np.array(poly).reshape(-1, 2).astype(np.int32)

        # Fill polygon
        cv2.fillPoly(mask, [points], instance_id)

    return mask

# Storage for dataset
images_list = []
annotations_list = []
img_id = 0
total_processed = 0
total_skipped = 0
total_instances = 0

# Process each city
for city in sorted(os.listdir(GT_ROOT)):
    city_gt = os.path.join(GT_ROOT, city)
    city_img = os.path.join(IMG_ROOT, city)
    city_pan = os.path.join(PAN_ROOT, city)
    city_sem = os.path.join(SEM_ROOT, city)

    if not os.path.isdir(city_gt):
        continue

    # Create output directories for this city
    os.makedirs(city_pan, exist_ok=True)
    os.makedirs(city_sem, exist_ok=True)

    print(f"\nProcessing city: {city}")

    # Process each JSON file in this city
    for json_file in sorted(os.listdir(city_gt)):
        if not json_file.endswith('_gtFine_polygons.json'):
            continue

        # Check limit
        if LIMIT is not None and total_processed >= LIMIT:
            break

        # Extract base name: frame1979_gtFine_polygons.json -> frame1979
        base_name = json_file.replace('_gtFine_polygons.json', '')

        # Corresponding image file
        img_name = base_name + '_leftImg8bit.jpg'
        img_path = os.path.join(city_img, img_name)

        if not os.path.exists(img_path):
            print(f"  ⚠️  Skip: {img_name} not found")
            total_skipped += 1
            continue

        # Load image to get dimensions
        img = Image.open(img_path)
        img_width, img_height = img.size

        # Load JSON annotations
        json_path = os.path.join(city_gt, json_file)
        with open(json_path, 'r') as f:
            gt_data = json.load(f)

        # Create panoptic mask (each instance gets unique ID)
        panoptic_mask = np.zeros((img_height, img_width), dtype=np.uint32)
        segments_info = []
        instance_id = 1

        # Process each object in the JSON
        for obj in gt_data.get('objects', []):
            # Get polygon coordinates
            polygon = obj.get('polygon', [])

            if len(polygon) < 3:  # Need at least 3 points
                continue

            # Convert polygon to mask for this instance
            instance_mask = polygon_to_mask([polygon], img_height, img_width, instance_id)

            # Add to panoptic mask
            panoptic_mask[instance_mask == instance_id] = instance_id

            # Calculate bbox and area
            ys, xs = np.where(instance_mask == instance_id)
            if len(xs) == 0:
                continue

            x0, x1 = int(xs.min()), int(xs.max())
            y0, y1 = int(ys.min()), int(ys.max())
            area = int(len(xs))

            # Add segment info
            segments_info.append({
                'id': int(instance_id),
                'category_id': 1,  # All objects are category 1
                'area': area,
                'bbox': [x0, y0, x1 - x0 + 1, y1 - y0 + 1],
                'iscrowd': 0
            })

            instance_id += 1
            total_instances += 1

        # Create semantic mask (all instances of same class get same ID)
        semantic_mask = np.zeros((img_height, img_width), dtype=np.uint8)
        for seg in segments_info:
            inst_id = seg['id']
            cat_id = seg['category_id']
            semantic_mask[panoptic_mask == inst_id] = cat_id

        # Save panoptic mask
        pan_name = base_name + '_leftImg8bit.png'
        pan_path = os.path.join(city_pan, pan_name)
        Image.fromarray(panoptic_mask).save(pan_path)

        # Save semantic mask
        sem_name = base_name + '_leftImg8bit.png'
        sem_path = os.path.join(city_sem, sem_name)
        Image.fromarray(semantic_mask).save(sem_path)

        # Add to dataset
        images_list.append({
            'id': img_id,
            'file_name': city + '/' + img_name,
            'height': img_height,
            'width': img_width
        })

        annotations_list.append({
            'image_id': img_id,
            'file_name': city + '/' + pan_name,
            'segments_info': segments_info
        })

        total_processed += 1
        if total_processed % 10 == 0:
            print(f"  ✓ Processed {total_processed} images, {total_instances} instances...")

        img_id += 1

    if LIMIT is not None and total_processed >= LIMIT:
        break

# Create COCO panoptic format JSON
panoptic_json = {
    'images': images_list,
    'annotations': annotations_list,
    'categories': [
        {
            'id': 0,
            'name': 'background',
            'supercategory': 'void',
            'isthing': 0,
            'color': [0, 0, 0]
        },
        {
            'id': 1,
            'name': 'object',
            'supercategory': 'thing',
            'isthing': 1,
            'color': [220, 20, 60]
        }
    ]
}

# Save JSON
with open(JSON_OUT, 'w') as f:
    json.dump(panoptic_json, f, indent=2)

print(f"\n{'='*70}")
print(f"✅ MASK GENERATION COMPLETE!")
print(f"{'='*70}")
print(f"Images processed:  {total_processed}")
print(f"Images skipped:    {total_skipped}")
print(f"Total instances:   {total_instances}")
print(f"Avg per image:     {total_instances/max(total_processed,1):.1f}")
print(f"\nFiles created:")
print(f"  Panoptic masks → {PAN_ROOT}")
print(f"  Semantic masks → {SEM_ROOT}")
print(f"  JSON metadata  → {JSON_OUT}")
print(f"{'='*70}")


CREATING PANOPTIC & SEMANTIC MASKS FROM POLYGON JSON

Processing city: 201


/tmp/ipython-input-1117777425.py:140: DeprecationWarning: Saving I mode images as PNG is deprecated and will be removed in Pillow 13 (2026-10-15)
  Image.fromarray(panoptic_mask).save(pan_path)


  ✓ Processed 10 images, 811 instances...

Processing city: 202

Processing city: 203
  ✓ Processed 20 images, 1290 instances...
  ✓ Processed 30 images, 1634 instances...

Processing city: 204
  ✓ Processed 40 images, 2302 instances...
  ✓ Processed 50 images, 3129 instances...

✅ MASK GENERATION COMPLETE!
Images processed:  50
Images skipped:    0
Total instances:   3129
Avg per image:     62.6

Files created:
  Panoptic masks → /content/idd_panoptic/train
  Semantic masks → /content/idd_semantic/train
  JSON metadata  → /content/panoptic_train.json


DATA MAPPING

In [ ]:
import torch
import copy
import torch
import numpy as np
from detectron2.data import detection_utils as utils
from detectron2.data import transforms as T
from detectron2.structures import BitMasks

class PanopticDataMapper:
    """
    Custom data mapper that loads images, semantic masks, and annotations.
    """

    def __init__(self, cfg, is_train=True):
        self.is_train = is_train
        self.image_format = cfg.INPUT.FORMAT

        # Build augmentations
        if is_train:
            self.augmentations = T.AugmentationList([
                T.ResizeShortestEdge(
                    cfg.INPUT.MIN_SIZE_TRAIN,
                    cfg.INPUT.MAX_SIZE_TRAIN,
                    cfg.INPUT.MIN_SIZE_TRAIN_SAMPLING
                ),
                T.RandomFlip(prob=0.5, horizontal=True, vertical=False)
            ])
        else:
            self.augmentations = T.AugmentationList([
                T.ResizeShortestEdge(
                    cfg.INPUT.MIN_SIZE_TEST,
                    cfg.INPUT.MAX_SIZE_TEST,
                    "choice"
                )
            ])

    def __call__(self, dataset_dict):
        """
        Args:
            dataset_dict: metadata from the dataset

        Returns:
            dict with image, sem_seg, and instances
        """
        dataset_dict = copy.deepcopy(dataset_dict)

        # 1. Load image
        image_path = os.path.join(IMG_ROOT, dataset_dict['file_name'])
        image = utils.read_image(image_path, format=self.image_format)
        utils.check_image_size(dataset_dict, image)

        # 2. Load semantic segmentation mask
        sem_file = dataset_dict['file_name'].replace('.jpg', '.png')
        sem_path = os.path.join(SEM_ROOT, sem_file)

        if not os.path.exists(sem_path):
            raise FileNotFoundError(f"Semantic mask not found: {sem_path}")

        sem_seg_gt = utils.read_image(sem_path, "L").squeeze(2)
        # 2B. Load panoptic mask (instance ids)
        pan_file = dataset_dict['pan_seg_file_name']
        pan_path = os.path.join(PAN_ROOT, pan_file)
        pan_mask = utils.read_image(pan_path, "L").squeeze(2)

        # 3. Apply augmentations
        aug_input = T.AugInput(image, sem_seg=sem_seg_gt)
        transforms = self.augmentations(aug_input)
        image = aug_input.image
        sem_seg_gt = aug_input.sem_seg

        # 4. Convert to tensors
        image_shape = image.shape[:2]
        dataset_dict['image'] = torch.as_tensor(
            np.ascontiguousarray(image.transpose(2, 0, 1))
        )
        dataset_dict['sem_seg'] = torch.as_tensor(
            sem_seg_gt.astype('long')
        )

        # 5. Process instance annotations + masks
        if 'annotations' in dataset_dict:
            annos = [
                utils.transform_instance_annotations(
                    obj, transforms, image_shape
                )
                for obj in dataset_dict.pop('annotations')
                if obj.get('iscrowd', 0) == 0
            ]

            instances = utils.annotations_to_instances(annos, image_shape)

            # ---- ADD INSTANCE MASKS ----
            masks = []
            for i in range(len(annos)):
                inst_id = i + 1
                binary_mask = (pan_mask == inst_id).astype("uint8")
                masks.append(torch.from_numpy(binary_mask))

            if len(masks) > 0:
                instances.gt_masks = BitMasks(torch.stack(masks))
            # ----------------------------

            dataset_dict['instances'] = utils.filter_empty_instances(instances)


        return dataset_dict

print("✓ Custom data mapper defined")

✓ Custom data mapper defined


REGISTER DATASET

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog

def load_panoptic_dataset():
    """
    Load the panoptic dataset from JSON.
    Returns list of dataset dicts in Detectron2 format.
    """
    with open(JSON_OUT, 'r') as f:
        data = json.load(f)

    dataset_dicts = []

    for img_info, ann_info in zip(data['images'], data['annotations']):
        record = {
            'file_name': img_info['file_name'],
            'image_id': img_info['id'],
            'height': img_info['height'],
            'width': img_info['width'],
            'pan_seg_file_name': ann_info['file_name'],
            'segments_info': ann_info['segments_info']
        }

        # Convert segments to annotations for instance detection
        annotations = []
        for seg in ann_info['segments_info']:
            if seg['category_id'] > 0:  # Skip background
                annotations.append({
                    'bbox': seg['bbox'],
                    'bbox_mode': 0,  # XYXY_ABS mode
                    'category_id': seg['category_id'] - 1,  # 0-indexed (1->0)
                    'iscrowd': seg.get('iscrowd', 0)
                })

        record['annotations'] = annotations
        dataset_dicts.append(record)

    return dataset_dicts

# Register dataset
dataset_name = "idd_panoptic_train"
DatasetCatalog.register(dataset_name, load_panoptic_dataset)
MetadataCatalog.get(dataset_name).set(
    thing_classes=["object"],
    stuff_classes=["background"],
    thing_dataset_id_to_contiguous_id={1: 0},
    stuff_dataset_id_to_contiguous_id={0: 0},
    image_root=IMG_ROOT,
    panoptic_root=PAN_ROOT,
    sem_seg_root=SEM_ROOT,
    panoptic_json=JSON_OUT,
    evaluator_type="coco_panoptic_seg",
)

print(f"✓ Dataset registered: {dataset_name}")

# Verify dataset is not empty
dataset_dicts = DatasetCatalog.get(dataset_name)
print(f"  Dataset size: {len(dataset_dicts)} images")
if len(dataset_dicts) == 0:
    raise ValueError("Dataset is empty! Check CELL 4 for errors.")

✓ Dataset registered: idd_panoptic_train
  Dataset size: 50 images


CONFIGURE MODEL

In [ ]:
from detectron2.config import get_cfg
from detectron2 import model_zoo

cfg = get_cfg()
cfg.merge_from_file(
    model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_50_3x.yaml")
)

# Dataset
cfg.DATASETS.TRAIN = (dataset_name,)
cfg.DATASETS.TEST = ()

# Dataloader
cfg.DATALOADER.NUM_WORKERS = 2

# Solver
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 100
cfg.SOLVER.STEPS = []
cfg.SOLVER.CHECKPOINT_PERIOD = 50

# Model
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # 1 thing class
cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 2  # background + object

# Output
cfg.OUTPUT_DIR = OUTPUT_DIR

print("="*70)
print("✓ Model configured")
print("="*70)
print(f"Dataset:           {cfg.DATASETS.TRAIN}")
print(f"Batch size:        {cfg.SOLVER.IMS_PER_BATCH}")
print(f"Learning rate:     {cfg.SOLVER.BASE_LR}")
print(f"Max iterations:    {cfg.SOLVER.MAX_ITER}")
print(f"ROI classes:       {cfg.MODEL.ROI_HEADS.NUM_CLASSES}")
print(f"Semantic classes:  {cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES}")
print(f"Output directory:  {cfg.OUTPUT_DIR}")
print("="*70)

✓ Model configured
Dataset:           ('idd_panoptic_train',)
Batch size:        2
Learning rate:     0.00025
Max iterations:    100
ROI classes:       1
Semantic classes:  2
Output directory:  /content/output


TRAINER

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.data import build_detection_train_loader

class PanopticTrainer(DefaultTrainer):
    """
    Custom trainer that uses PanopticDataMapper.
    """

    @classmethod
    def build_train_loader(cls, cfg):
        mapper = PanopticDataMapper(cfg, is_train=True)
        return build_detection_train_loader(cfg, mapper=mapper)

print("✓ Custom trainer defined")

✓ Custom trainer defined


TRAINING

In [ ]:
print("\n" + "="*70)
print("🚀 STARTING TRAINING")
print("="*70)

trainer = PanopticTrainer(cfg)
trainer.resume_or_load(resume=False)

# Train
trainer.train()

print("\n" + "="*70)
print("✅ TRAINING COMPLETE!")
print("="*70)
print(f"Model saved to: {cfg.OUTPUT_DIR}")
print(f"Check for model_final.pth")
print("="*70)


🚀 STARTING TRAINING
[02/03 14:02:26 d2.engine.defaults]: Model:
PanopticFPN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
 

backbone.fpn_lateral2.{bias, weight}
backbone.fpn_lateral3.{bias, weight}
backbone.fpn_lateral4.{bias, weight}
backbone.fpn_lateral5.{bias, weight}
backbone.fpn_output2.{bias, weight}
backbone.fpn_output3.{bias, weight}
backbone.fpn_output4.{bias, weight}
backbone.fpn_output5.{bias, weight}
proposal_generator.rpn_head.anchor_deltas.{bias, weight}
proposal_generator.rpn_head.conv.{bias, weight}
proposal_generator.rpn_head.objectness_logits.{bias, weight}
roi_heads.box_head.fc1.{bias, weight}
roi_heads.box_head.fc2.{bias, weight}
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.deconv.{bias, weight}
roi_heads.mask_head.mask_fcn1.{bias, weight}
roi_heads.mask_head.mask_fcn2.{bias, weight}
roi_heads.mask_head.mask_fcn3.{bias, weight}
roi_heads.mask_head.mask_fcn4.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}
sem_seg_head.p2.0.norm.{bias, weight}
sem_seg_head.p2.0.weight
sem_seg_head.p3.0.norm.{bias, weight}


[02/03 14:02:26 d2.engine.train_loop]: Starting training from iteration 0
[02/03 14:02:55 d2.utils.events]:  eta: 0:01:54  iter: 19  total_loss: 3.826  loss_sem_seg: 0.3491  loss_rpn_cls: 0.6887  loss_rpn_loc: 1.348  loss_cls: 0.5096  loss_box_reg: 0.03036  loss_mask: 0.688    time: 1.4322  last_time: 1.3976  data_time: 0.6117  last_data_time: 0.5133   lr: 4.7703e-05  max_mem: 5918M
[02/03 14:03:21 d2.utils.events]:  eta: 0:01:17  iter: 39  total_loss: 3.701  loss_sem_seg: 0.4231  loss_rpn_cls: 0.6832  loss_rpn_loc: 1.22  loss_cls: 0.3396  loss_box_reg: 0.02605  loss_mask: 0.6213    time: 1.3555  last_time: 1.4998  data_time: 0.4762  last_data_time: 0.6001   lr: 9.7653e-05  max_mem: 5936M
[02/03 14:03:50 d2.utils.events]:  eta: 0:00:51  iter: 59  total_loss: 3.575  loss_sem_seg: 0.6274  loss_rpn_cls: 0.6598  loss_rpn_loc: 1.354  loss_cls: 0.2785  loss_box_reg: 0.03731  loss_mask: 0.508    time: 1.3612  last_time: 1.3430  data_time: 0.5164  last_data_time: 0.4800   lr: 0.0001476  max_me

VERIFY

In [ ]:
import glob

print("\n" + "="*70)
print("TRAINING OUTPUTS")
print("="*70)

# List all files in output directory
output_files = os.listdir(cfg.OUTPUT_DIR)
print(f"\nFiles in {cfg.OUTPUT_DIR}:")
for f in sorted(output_files):
    fpath = os.path.join(cfg.OUTPUT_DIR, f)
    if os.path.isfile(fpath):
        size_mb = os.path.getsize(fpath) / (1024 * 1024)
        print(f"  {f:<30} {size_mb:>8.2f} MB")

# Check for final model
model_files = glob.glob(os.path.join(cfg.OUTPUT_DIR, "model_*.pth"))
if model_files:
    print(f"\n✓ Found {len(model_files)} model checkpoint(s)")
else:
    print("\n⚠️  No model checkpoints found!")

print("="*70)


TRAINING OUTPUTS

Files in /content/output:
  events.out.tfevents.1770126713.75f14d711bae.1168.0     0.00 MB
  events.out.tfevents.1770127375.75f14d711bae.1168.1     0.01 MB
  last_checkpoint                    0.00 MB
  metrics.json                       0.00 MB
  model_0000049.pth                347.16 MB
  model_0000099.pth                347.16 MB
  model_final.pth                  347.16 MB

✓ Found 3 model checkpoint(s)


DELETE

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog

dataset_name = "idd_panoptic_train"

# Remove old registration if it exists
if dataset_name in DatasetCatalog.list():
    DatasetCatalog.remove(dataset_name)
    MetadataCatalog.remove(dataset_name)
